# Resolvendo o Problema do **Caixeiro Viajante** usando _`Google Maps`_

> **MVP**:

In [1]:
from modules import io  # entrada e saída
from modules import browser  # navegador
from modules import navigation  # navegações
import json
from modules import processing
from selenium.webdriver.common.keys import Keys

In [5]:
# link padrão
url = 'https://www.google.com/maps'

# ponto de origem
point_origin = 'uberlândia, mg'

# tipo de condução
type_driving = 'carro'

# localidades
places = 'capitals'

# capitais
locations = io.touch(places + '.txt')

# inserindo o ponto de origem
locations.insert(0, point_origin)

# instanciando o driver
driver = browser.create_driver(url)

# setando o primeiro destino, para abrir as rotas
navigation.set_destination(locations[0], driver, box=1)

# abrindo a seção de rotas
navigation.click_routes_button(driver)

# selecionando o estilo de deslocamento
navigation.set_driving(driver, type_driving)

# realizando a leitura dos arquivos já processados
data = io.touch(places + '.txt', 'output', 'r', True)

# extraindo as chaves já processadas
already_processed = [list(key.keys())[0] for key in data]

# abrindo o arquivo de output
with open(fr'io/output/{places}.txt', 'a', encoding='utf-8') as file:

    # percorrendo os endereços
    for a, address_a in enumerate(locations):

        # percorrendo os endereços
        for b, address_b in enumerate(locations):

            # as localizações devem ser diferentes entre si e, não ter sido processada anteriormente
            if a != b and f'{a}_{b}' not in already_processed:
                print(f'{address_a}, {address_b}')

                # adicionando os dados de busca
                navigation.set_destination(address_a, driver, 1)
                navigation.set_destination(address_b, driver, 2)

                # raspando as informações
                time = browser.get_time(driver)
                distance = browser.get_distance(driver)

                # estruturando os dados
                data = {
                    f'{a}_{b}': {
                        'address_a': address_a,
                        'address_b': address_b,
                        'time': time,
                        'distance': distance,
                    }
                }

                # salvando os dados da linha
                file.write(json.dumps(data, ensure_ascii=False) + '\n')

                # limpando memória
                file.flush()

# realizando uma nova leitura, com dados atualizados
data = io.touch(places + '.txt', 'output', 'r', True)

# extraindo os dados
extract_data = [processing.extract_info(value) for value in data]

# estrutura de dados
extract_data = {key: value for item in extract_data for key, value in item.items()}

# caldulando otimização do tsp
solution = processing.tsp(locations, extract_data)

# realizando a plotagem dos resultados
navigation.plot_routes(locations, solution, driver)